# Model Free Prediction

- prediction
  - value를 학습하는 것
  - value를 찾는 것이고
- control
  - 더 나은 policy를 찾는 것
  - value function을 최적화 하는 것

- value를 찾을려면 어떤 policy를 따라갔을 때 게임이 끝났을 때 얻을 return의 기대값, policy가 정해져 있어야한다.
- policy가 정해져있다 가정하고 에피소드가 끝났을 때 return을 얼마 받을지

## Monte-Carlo Reinforcement Learning

- Monte-Carlo, 직접 구하기 어려운 것을 사건을 실행하면서 나오는 실제 값을 통해서 추정하는 것이다. 해보고 배우는 것이다.
- 게임을 끝까지 해보고 나온 return을 저장해 뒀다가 그것을 평균낸다.

- MDP의 transition과 reward를 몰라도 policy를 따라 하면되니까 model-free다.
- 에피소드가 끝나야 return(게임이 끝날때 까지 얻은 reward의 축적된 합이다.)이 정해지기 때문에 return을 평균낸게 value다.

## Monte-Carlo (Policy Evaluation = Prediction)

- 목적, episode로 부터 \\(v_{\pi}\\)를 배운다.

- value function은 return의 기대값이다.
$$ v_{\pi}(s) = \mathbb{E}_{\pi}[G_t\;|\;S_t=s] $$
- return은 확률변수이다. 매번 에피소드를 진행할 때 마다 다르다. 같은 policy를 하더라도 어떤 return을 받을지 매번 다르다.

- 그래서 monte-carlo evaluation은 return의 empirical mean을 사용한다.

## First-Visit Monte-Carlo Policy Evaluation

- 한 에피소드에서 여러번 하나의 state를 방문하는 경우가 생긴다. 이때 First visit monte carlo는 처음 방문한 것만 적용되는 방식이다.
- 큰수의 법칙에 의해서 n이 무한대로 가면 결국 수렴한다.

## Every-Visit Monte-Carlo Policy Evaluation

- 여러번 방문을 모두 방문으로 사용하는 경우이다. 위와 비교했을 때 다르다.

- 알고리즘
  - Increment counter, \\(N(s) \leftarrow N(s) + 1\\)
  - Increment total return \\(S(s) \leftarrow S(s) + G_t\\)
  - value is estimated by mean return \\(V(s) = \frac{S(s)}{N(s)}\\)
  - Again, \\(V(s) \rightarrow v_{\pi}(s)\\) as \\(N(s) \rightarrow \infty\\)

- 하나의 가정이 있는데, 방문 하지 못하는 state는 없다는 가정이 있다.

## Incremental Mean

- 잘 생각해보면 각 state마다 평균값을 다 저장하고 있어야 한다. 왜냐하면 에피소드가 끝날때 까지 기다렸다가 reward의 평균을 계산하기 때문이다.
- 그래서 Incremental Mean을 사용하면 저장할 필요 없이 교정하면 된다.

- 알고리즘
  - \\(N(S_t) \leftarrow N(S_t) + 1\\)
  - \\(V(S_t) \leftarrow + \frac{1}{N(S_t)}(G_t - V(S_t))\\)

- 다시 생각해보면,
$$ V(S_t) \leftarrow V(S_t) + \frac{1}{N(S_t)}(G_t - V(S_t)) $$
- 이전 값이 있고 \\(V_(S_t)\\),
- 지금 나온 error가 있다. 여기서 error는 현재 나온 \\(G_t\\)와 \\(V(S_t)\\)차이가 error이다. error만큼 update해준다. 라고 볼 수 있음.
- \\(N(S_t)\\)가 분모에 있기 때문에 N이 커질수록 값은 작아진다 따라서 값을 고정 시킬수도 있는데 그 이유는, N이 분모로 가게 되면 최신 기억은 update가 많이 되고 나중 기억은 update가 적게 될 수 밖에 없다. 따라서 최신 기억을 잘 update 하기 위해서 아래 식을 사용한다.
$$ V(S_t) \leftarrow V(S_t) + \alpha (G_t - V(S_t)) $$

## Temporal-Difference Learning

- TD와 MC의 차이는 에피소드가 끝나지 않아도 배울 수 있다.
- TD는 guess로 guess를 update한다.
- 그렇다면, TD는 무슨값으로 update하길래 에피소드가 끝나지 않아도 update가 가능한가?

### Every visit Monte-Carlo

- \\(G_t\\)의 방향으로 \\(V(S_t)\\)를 학습한다.
$$ V(S_t) \leftarrow V(S_t) + \alpha(G_t - V(S_t)) $$

### Simplest temporal-difference learning algorithm: TD(0)

- \\(R_{t+1} + \gamma V(S_{t+1})\\)의 방향으로 \\(V(S_t)\\)를 update 하는 것이다.
$$ V(S_t) \leftarrow V(S_t) + \alpha(R_{t+1} + \gamma V(S_{t+1}) - V(S_t)) $$
- TD target, \\(R_{t+1} + \gamma V(S_{t+1}) \\)
  - 현재 state \\(S_t\\)가 있는데 \\(S_t\\)에서 앞으로 reward를 얼마 받을지 예측하는게 \\(V(S_t)\\)인데 여기서 한 step을 가본다. 그러면 \\(S_{t+1}\\)이 되고 \\(S_{t+1}\\)에서 예측치가 있다. 그렇다면 한 step가서 예측하는 것이 더 정확하기 때문에 그 방향으로 V를 update한다.
  - 예를 들어) 내가 차를 타고 가는데 중앙선을 넘었을 때 화물차와 부딪히지 않은 상황을 가정해보자 만약 MC였다면 결국 최종 reward는 0이기 때문에 분명 위험 상황이었음에도 불구하고 나는 reward를 0을 받는다. 하지만 TD의 경우 죽을 수도 있었던 위험 상황인 것을 step 단위로 인지 하기 때문에 reward를 음수 값으로 받을 수 있다.
- TD error, \\(\delta_t = R_{t+1} + \gamma V(S_{t+1} - V(S_t) \\)

- 또한, MC와 TD는 update 방향과 언제 update하는 지에 대한 차이 인것이지 어느것이 더 많이 update되는 것은 아니다.

- 왜 temporal difference냐?
  - 순간적인 차이를 이용해서 학습한다.
  - 현재 guess를 update하기 위해서 한 step 더 간 guess를 이용한다.
- 예측치로 예측치를 update하면? 예측치가 틀리면?
  - 틀리더라도 결과적으로 수렴한다.

## Bias/Variance Trade-Off

- Return \\(G_t\\)는 \\(v_{\pi}(S_t)\\)에 unbiased estimate이다.
  - \\(G_t\\)의 기대 값이 \\(v_{\pi}(S_t)\\)이다. 따라서 \\(G_t\\)를 계속 sampling하면 평균이 \\(v_{\pi}\\)로 수렴한다. 따라서 편향이 없게 된다.

- True TD target \\(R_{t+1} + \gamma v_{\pi}(S_{t+1})\\)
  - 전지전능한 신이 있어서 \\(v_{\pi}(S_{t+1})\\)의 값을 알려주면, TD target은 unbiased estimate이다.
  - bellman equation으로 보장가능
- But, 우리는 신이 아니기 때문에 \\(v_{\pi}(S_{t+1})\\)의 실제값을 알지 못한다. 그래서 biased 되어 있다. 그래서 행동을 100만번 한다고 해서 \\(v_{\pi}\\)가 된다는 보장이 없다.
- 그래서 bias 관점에서는 MC보다 TD가 안좋다.

- 하지만 variance 관점에서 보면,
  - TD target은 variance가 MC보다 훨씬 낮다.
  - TD는 한 step만 보기 때문에 작고, MC는 한번의 에피소드를 보기 때문에 variance가 크다.

- Q, bias가 있으면 알고리즘이 동작을 하나?
  - 동작한다고 한다. 하지만 증명은 없다.

## 여러 \\(\alpha\\)에 대해서 차이

- 에피소드가 진행됨에 따라 RMS error 구한것
- MC는 완만하게 줄어드는 반면,
- TD는 줄어들었다가 늘었다가 한다. 다시 늘어나는 이유는 \\(\alpha\\)가 너무 커서이다.

## Batch MC and TD

- 무한적으로 에피소드를 뽑을 수 없을 경우, 제한된 에피소드가 있을 때 과연 MC와 TD가 같은 곳으로 수렴을 하나? 두 가지 방법 모두 수렴은 할까?

## MC, TD 차이

- TD는 exploits Markov property
- MC does not exploit Markov property

## Monte-Carlo Backup

$$ V(S_t) \leftarrow V(S_t) + \alpha(G_t - V(S_t)) $$

## Temporal-Difference Backup

- bootstrapping, 끝까지 안해보고 추측치로 추측치를 update하는 것이 bootstrapping
$$ V(S_t) \leftarrow V(S_t) + \alpha(R_{t+1} + \gamma V(S_{t+1}) - V(S_t)) $$

## Dynamic Programming Backup

$$ V(S_t) \leftarrow \mathbb{E}_{\pi}[R_{t+1} + \gamma V(S_{t+1})] $$

## Bootstrapping, Sampling 차이

- bootstrapping,
  - 추측치로 추측치를 update하는 것이기 때문에 update에 추측치가 포함이 됨.
- sampling,
  - sample을 가지고 update한다.

## n-Step Prediction

- 1-step을 보는 것이 TD(0)이고,
- n-step을 보는 것이 MC이다.

## n-Step Return

- n 만큼은 reward를 넣고, \\(S_{t+2}\\)에서는 추측치를 넣고

<img width="452" alt="2018-11-24 5 01 26" src="https://user-images.githubusercontent.com/22078438/48965930-bf064b00-f00a-11e8-9e63-4dfc5c52ad09.png">

#### 결론적으로

$$ G_t^{(n)} = R_{t+1} + \gamma R_{t+2} + ... + \gamma^{n-1}R_{t+n} + \gamma^n V(S_{t+n}) $$

$$ V(S_t) \leftarrow V(S_t) + \alpha( G_t^{(n)} - V(S_t)) $$

## Large Random Walk Example

- 그래프를 보면 1이 가장 좋지도 않고, 1000이 가장 좋지도 않다.
- 그래서 3, 4, 5가 가장좋다.

## Averaging n-Step Returns

- 각 step의 평균을 사용하면 안되는가?

## \\(\lambda \\)-return

- TD(0) 부터 MC까지 모든 것을 평균 내도 된다.
- 이것이 TD(\\(\lambda\\))이다.


<img width="311" alt="2018-11-24 5 10 48" src="https://user-images.githubusercontent.com/22078438/48966009-004b2a80-f00c-11e8-9591-8f43ba2e61d9.png">


- 위 무한 급수를 다 더하면 1이 된다.
- MC로 갈수록 가중치가 적게 들어간다.

## Forward view, Backward view 차이

- forward view, 내 미래를 보기 때문에 forward view이다.
- 시간이 갈수록 값이 줄어든다.
- 왜 geometric mean을 사용하냐? -> 계산적으로 효율적이다. 이전 것을 저장하지 않고 바로바로 update가능하기 때문
$$ G^{\lambda}_t = (1 - \lambda)\sum^{\infty}_{n=1} \lambda^{n-1} G^{(n)}_t $$
- 게임이 끝나야 update 가능하다. MC와 비슷하다.

## backward view

- 매 스텝마다 update가능하다.

### Eligibility Traces

- 어떤 사건이 일어나면 책임을 묻는데, 책임이 클수록 update 많이 해주는 방식
- ex)
  - 종이 3번 울리고 전구가 한번 켜지고 전기 충격을 받는 상황에서
  - 전구가 전기충격 바로 직전에 일어났기 때문에 책임을 물을수도 있고,
  - 종이 3번 울려서 많이 울렸기 때문에 전기 충격을 받았다고 책임을 물을수도 있다.
  - 그래서 Frequency heuristic, Recency heuristic
    - 많이 일어난 것에 책임을 많이 묻고, 최근에 일어난 일에 대해서 책임을 많이 물어야한다.
- 방문하게 되면 1을 주고, 안방문 할때마다 조금씩 줄인다.


<img width="306" alt="2018-11-24 5 20 20" src="https://user-images.githubusercontent.com/22078438/48966086-3e951980-f00d-11e8-83d8-dd4650399a68.png">

- 위 그림에서 \\((S_t = s)\\)의 의미는 방문 했을 때 1을 주고 아니면 0을 주는 것이다.

- \\(E_t(s)\\), 어느 시점 t일때 s에 대한 eligibility trace값
- 여기서 \\(\alpha\\)는 every visit, first visit에서 사용된 것 처럼 몇번 그 state에 방문했는가?를 \\(\alpha\\)로 사용한다. 왜냐하면, 최근 방문의 error값이 작아지기 때문에 \\(\alpha\\)를 둬서 최근 방문이 중요하도록 만든다.
- 이 state의 책임을 기록해두는 값이 있어서 그것을 곱해서 그것만큼 update해주면, 이게 TD(\\(\lambda\\)와 동일하다.

## Online, Offline policy 차이

- online, 학습하면서 환경에서 움직임. 1-step학습해서 움직이고, ...
- offline, 다 움직이고 에피소드 끝나면 학습한다.